**Analytics & Dashboard KPIs**

In [0]:
# MAGIC %md
# MAGIC # IM8 Health – Analytics & Dashboard KPIs
# MAGIC ## Customer Health & Subscription Insights Dashboard

# COMMAND ----------

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *

database_name = "im8_health"

# Load all tables
df_customers = spark.table(f"{database_name}.dim_customers")
df_products = spark.table(f"{database_name}.dim_products")
df_subscriptions = spark.table(f"{database_name}.fact_subscriptions")
df_orders = spark.table(f"{database_name}.fact_orders")
df_engagement = spark.table(f"{database_name}.fact_engagement")
df_health = spark.table(f"{database_name}.fact_health_outcomes")

print("✅ All tables loaded")

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 1: EXECUTIVE SUMMARY METRICS
# MAGIC **Purpose:** Top-line health of the business at a glance for leadership

# COMMAND ----------

# Current period metrics (latest full month = Jan 2026)
# We calculate for the most recent complete month

latest_month = "2026-01"
previous_month = "2025-12"

# --- Total Customers ---
total_customers = df_customers.count()

# --- Active Subscribers ---
active_subs = df_subscriptions.filter(F.col("status") == "active").count()

# --- Current MRR ---
current_mrr = df_subscriptions.filter(F.col("status") == "active").agg(
    F.round(F.sum("mrr_usd"), 2).alias("current_mrr")
).collect()[0]["current_mrr"]

# --- ARR ---
current_arr = current_mrr * 12

# --- Total Revenue to Date ---
total_revenue = df_orders.agg(F.round(F.sum("order_total_usd"), 2)).collect()[0][0]

# --- Average Order Value ---
avg_order_value = df_orders.agg(F.round(F.avg("order_total_usd"), 2)).collect()[0][0]

# --- Overall Churn Rate ---
total_subs_ever = df_subscriptions.count()
churned_count = df_subscriptions.filter(F.col("status") == "cancelled").count()
overall_churn_rate = round(churned_count / total_subs_ever * 100, 1)

# --- Net Revenue Retention ---
# Simplified: (Current MRR from cohorts that existed 12 months ago) / (Their MRR 12 months ago)

# Display executive summary
exec_summary = spark.createDataFrame([{
    "metric": "Total Customers Acquired",
    "value": f"{total_customers:,}",
    "context": "Since Dec 2024 launch"
}, {
    "metric": "Active Subscribers",
    "value": f"{active_subs:,}",
    "context": f"{active_subs/total_customers*100:.1f}% of total customers"
}, {
    "metric": "Monthly Recurring Revenue (MRR)",
    "value": f"${current_mrr:,.2f}",
    "context": "Current active subscriptions"
}, {
    "metric": "Annualized Run Rate (ARR)",
    "value": f"${current_arr:,.2f}",
    "context": "MRR × 12"
}, {
    "metric": "Total Revenue",
    "value": f"${total_revenue:,.2f}",
    "context": "All orders to date"
}, {
    "metric": "Average Order Value (AOV)",
    "value": f"${avg_order_value:.2f}",
    "context": "Across all order types"
}, {
    "metric": "Overall Churn Rate",
    "value": f"{overall_churn_rate}%",
    "context": f"{churned_count:,} of {total_subs_ever:,} subscriptions"
}])

exec_summary.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 2: COHORT RETENTION BY ACQUISITION MONTH
# MAGIC **Insight:** _Which acquisition cohorts retain best? Is retention improving over time as we refine our product and onboarding?_

# COMMAND ----------

# Build cohort retention matrix
# Join customers with subscriptions to get cohort (acquisition month) and subscription months

cohort_base = df_customers.join(
    df_subscriptions,
    "customer_id",
    "inner"
).select(
    F.col("customer_id"),
    F.col("acquisition_month").alias("cohort"),
    F.col("subscription_start_date"),
    F.col("cancel_date"),
    F.col("status"),
    F.col("months_active")
)

# Generate retention for each month after acquisition
# For each cohort, calculate what % of subscribers are still active at month N

from pyspark.sql.functions import explode, sequence, lit, expr

# Create a row per customer per month they were active
cohort_months = cohort_base.select(
    "customer_id",
    "cohort",
    "months_active",
    "status"
).withColumn("month_numbers", F.expr("sequence(1, months_active)")) \
 .select("customer_id", "cohort", "status", F.explode("month_numbers").alias("tenure_month"))

# Count cohort size and retained customers per month
cohort_sizes = cohort_base.groupBy("cohort").count().withColumnRenamed("count", "cohort_size")

cohort_retention = cohort_months.groupBy("cohort", "tenure_month").agg(
    F.countDistinct("customer_id").alias("retained_customers")
).join(cohort_sizes, "cohort") \
 .withColumn("retention_rate", F.round(F.col("retained_customers") / F.col("cohort_size") * 100, 1)) \
 .orderBy("cohort", "tenure_month")

cohort_retention.createOrReplaceTempView("cohort_retention")

# Pivot for dashboard display
cohort_pivot = cohort_retention.groupBy("cohort", "cohort_size").pivot("tenure_month").agg(
    F.first("retention_rate")
).orderBy("cohort")

cohort_pivot.createOrReplaceTempView("cohort_retention_pivot")
cohort_pivot.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ### 📈 Cohort Retention Trend Visualization

# COMMAND ----------

# For line chart - retention curves by cohort
# Select key cohorts for clarity
key_cohorts = cohort_retention.filter(
    F.col("cohort").isin("2024-12", "2025-01", "2025-03", "2025-06", "2025-09")
).orderBy("cohort", "tenure_month")

key_cohorts.createOrReplaceTempView("cohort_retention_trends")
key_cohorts.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 3: CHURN RISK SIGNALS
# MAGIC **Insight:** _What behaviors predict churn? Can we intervene before customers cancel?_

# COMMAND ----------

# Build customer-level churn risk model features
# Join all data sources to create a comprehensive customer profile

# Latest engagement per customer
latest_engagement = df_engagement.withColumn(
    "rn", F.row_number().over(Window.partitionBy("customer_id").orderBy(F.desc("month")))
).filter(F.col("rn") <= 3)  # Last 3 months

engagement_features = latest_engagement.groupBy("customer_id").agg(
    F.round(F.avg("app_opens"), 1).alias("avg_app_opens_last3m"),
    F.round(F.avg("engagement_score"), 1).alias("avg_engagement_last3m"),
    F.sum("surveys_completed").alias("surveys_last3m"),
    F.sum("referrals_made").alias("referrals_last3m"),
    F.sum("support_tickets").alias("support_tickets_last3m"),
    F.avg("nps_score").alias("avg_nps")
)

# Latest health outcomes
latest_health = df_health.withColumn(
    "rn", F.row_number().over(Window.partitionBy("customer_id").orderBy(F.desc("month")))
).filter(F.col("rn") == 1)

health_features = latest_health.select(
    "customer_id",
    F.col("overall_wellness_score").alias("latest_wellness_score"),
    "improvement_vs_baseline"
)

# Order frequency
order_features = df_orders.groupBy("customer_id").agg(
    F.count("order_id").alias("total_orders"),
    F.round(F.avg("order_total_usd"), 2).alias("avg_order_value"),
    F.max("order_date").alias("last_order_date"),
    F.datediff(F.lit("2026-01-31"), F.max("order_date")).alias("days_since_last_order")
)

# Build churn risk profile
churn_risk = df_subscriptions.filter(F.col("status") == "active") \
    .join(df_customers, "customer_id") \
    .join(engagement_features, "customer_id", "left") \
    .join(health_features, "customer_id", "left") \
    .join(order_features, "customer_id", "left") \
    .select(
        "customer_id",
        "customer_persona",
        "acquisition_channel",
        "region",
        "product_id",
        "plan_type",
        "mrr_usd",
        "months_active",
        "avg_app_opens_last3m",
        "avg_engagement_last3m",
        "surveys_last3m",
        "referrals_last3m",
        "support_tickets_last3m",
        "avg_nps",
        "latest_wellness_score",
        "improvement_vs_baseline",
        "total_orders",
        "avg_order_value",
        "days_since_last_order"
    )

# Calculate risk score (rule-based model)
churn_risk_scored = churn_risk.withColumn(
    "risk_score",
    F.when(F.col("avg_engagement_last3m") < 30, 30).otherwise(0) +
    F.when(F.col("avg_app_opens_last3m") < 5, 20).otherwise(0) +
    F.when(F.col("surveys_last3m") == 0, 15).otherwise(0) +
    F.when(F.col("support_tickets_last3m") > 0, 10).otherwise(0) +
    F.when(F.col("improvement_vs_baseline") == "Declined", 15).otherwise(
        F.when(F.col("improvement_vs_baseline") == "Stable", 5).otherwise(0)
    ) +
    F.when(F.col("days_since_last_order") > 45, 10).otherwise(0) +
    F.when(F.col("months_active") <= 2, 10).otherwise(
        F.when(F.col("months_active") <= 4, 5).otherwise(0)
    ) +
    F.when(F.col("avg_nps") < 7, 10).otherwise(0)
).withColumn(
    "risk_level",
    F.when(F.col("risk_score") >= 60, "🔴 Critical")
     .when(F.col("risk_score") >= 40, "🟠 High")
     .when(F.col("risk_score") >= 20, "🟡 Medium")
     .otherwise("🟢 Low")
).withColumn(
    "recommended_action",
    F.when(F.col("risk_score") >= 60, "Immediate outreach: personal call + offer")
     .when(F.col("risk_score") >= 40, "Triggered email sequence + survey")
     .when(F.col("risk_score") >= 20, "Engagement nudge: content + community invite")
     .otherwise("Continue nurture program")
)

churn_risk_scored.createOrReplaceTempView("churn_risk_scored")

# Summary
churn_risk_summary = churn_risk_scored.groupBy("risk_level").agg(
    F.count("customer_id").alias("customer_count"),
    F.round(F.sum("mrr_usd"), 2).alias("mrr_at_risk"),
    F.round(F.avg("months_active"), 1).alias("avg_tenure_months"),
    F.round(F.avg("avg_engagement_last3m"), 1).alias("avg_engagement")
).orderBy(F.desc("mrr_at_risk"))

churn_risk_summary.createOrReplaceTempView("churn_risk_summary")
churn_risk_summary.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ### 🔍 Churn Risk by Segment

# COMMAND ----------

# Churn risk by various dimensions for dashboard filters
# By Product
churn_by_product = churn_risk_scored.groupBy("product_id", "risk_level").agg(
    F.count("customer_id").alias("count"),
    F.round(F.sum("mrr_usd"), 2).alias("mrr_at_risk")
).orderBy("product_id", "risk_level")

churn_by_product.createOrReplaceTempView("churn_risk_by_product")
churn_by_product.display()

# By Channel
churn_by_channel = churn_risk_scored.groupBy("acquisition_channel", "risk_level").agg(
    F.count("customer_id").alias("count"),
    F.round(F.sum("mrr_usd"), 2).alias("mrr_at_risk")
).orderBy("acquisition_channel", "risk_level")

churn_by_channel.createOrReplaceTempView("churn_risk_by_channel")
churn_by_channel.display()

# By Region
churn_by_region = churn_risk_scored.groupBy("region", "risk_level").agg(
    F.count("customer_id").alias("count"),
    F.round(F.sum("mrr_usd"), 2).alias("mrr_at_risk")
).orderBy("region", "risk_level")

churn_by_region.createOrReplaceTempView("churn_risk_by_region")
churn_by_region.display()

# By Persona
churn_by_persona = churn_risk_scored.groupBy("customer_persona", "risk_level").agg(
    F.count("customer_id").alias("count"),
    F.round(F.sum("mrr_usd"), 2).alias("mrr_at_risk")
).orderBy("customer_persona", "risk_level")

churn_by_persona.createOrReplaceTempView("churn_risk_by_persona")
churn_by_persona.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ### 📋 Top Churn Risk Watchlist

# COMMAND ----------

# Top 20 customers at highest risk - for action
watchlist = churn_risk_scored.filter(F.col("risk_level").isin("🔴 Critical", "🟠 High")) \
    .orderBy(F.desc("risk_score"), F.desc("mrr_usd")) \
    .limit(20)

watchlist.createOrReplaceTempView("churn_watchlist")
watchlist.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 4: HEALTH OUTCOME CORRELATION WITH RETENTION
# MAGIC **Insight:** _Do customers who report health improvements retain longer? This is the most strategic insight for IM8 – proving product efficacy drives business outcomes._

# COMMAND ----------

# Correlate health improvement with subscription status
health_retention = df_health.select(
    "customer_id", "improvement_vs_baseline"
).dropDuplicates(["customer_id"]).join(  # Take latest assessment
    df_subscriptions.select("customer_id", "status", "months_active"),
    "customer_id",
    "inner"
)

# Retention by improvement category
retention_by_health = health_retention.groupBy("improvement_vs_baseline").agg(
    F.count("customer_id").alias("total_customers"),
    F.sum(F.when(F.col("status") == "active", 1).otherwise(0)).alias("still_active"),
    F.sum(F.when(F.col("status") == "cancelled", 1).otherwise(0)).alias("churned"),
    F.round(F.avg("months_active"), 1).alias("avg_months_active")
).withColumn(
    "retention_rate", F.round(F.col("still_active") / F.col("total_customers") * 100, 1)
).withColumn(
    "churn_rate", F.round(F.col("churned") / F.col("total_customers") * 100, 1)
).orderBy(F.desc("retention_rate"))

retention_by_health.createOrReplaceTempView("retention_by_health_outcome")
retention_by_health.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ### 📈 Health Scores Over Time (Active vs Churned)

# COMMAND ----------

# Compare health trajectories of active vs churned customers
health_trajectory = df_health.join(
    df_subscriptions.select("customer_id", "status"),
    "customer_id",
    "inner"
).groupBy("month", "status").agg(
    F.round(F.avg("energy_score"), 2).alias("avg_energy"),
    F.round(F.avg("sleep_score"), 2).alias("avg_sleep"),
    F.round(F.avg("digestion_score"), 2).alias("avg_digestion"),
    F.round(F.avg("overall_wellness_score"), 2).alias("avg_overall_wellness"),
    F.count("customer_id").alias("respondents")
).orderBy("month", "status")

health_trajectory.createOrReplaceTempView("health_trajectory")
health_trajectory.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ### 🏥 Health Improvement by Product

# COMMAND ----------

# Which product drives the best health outcomes?
health_by_product = df_health.join(
    df_subscriptions.select("customer_id", "product_id"),
    "customer_id",
    "inner"
).join(
    df_products.select("product_id", "product_name"),
    "product_id",
    "inner"
)

health_product_summary = health_by_product.groupBy("product_name", "improvement_vs_baseline").agg(
    F.count("customer_id").alias("responses")
)

# Calculate improvement rate per product
total_by_product = health_by_product.groupBy("product_name").agg(
    F.count("customer_id").alias("total_responses"),
    F.round(F.avg("overall_wellness_score"), 2).alias("avg_wellness_score")
)

improved_by_product = health_by_product.filter(F.col("improvement_vs_baseline") == "Improved") \
    .groupBy("product_name").agg(F.count("customer_id").alias("improved_responses"))

product_efficacy = total_by_product.join(improved_by_product, "product_name", "left") \
    .withColumn("improvement_rate", 
                F.round(F.col("improved_responses") / F.col("total_responses") * 100, 1)) \
    .orderBy(F.desc("improvement_rate"))

product_efficacy.createOrReplaceTempView("product_health_efficacy")
product_efficacy.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 5: REVENUE METRICS – MRR, LTV, Growth
# MAGIC **Insight:** _How is revenue growing? What's the lifetime value of different customer segments?_

# COMMAND ----------

# --- MRR Over Time ---
# Calculate MRR for each month based on active subscriptions at that point

# Create a month dimension
months = spark.sql("""
    SELECT DISTINCT month_date FROM (
        SELECT explode(sequence(to_date('2024-12-01'), to_date('2026-01-31'), interval 1 month)) as month_date
    ) t
    WHERE month_date <= '2026-01-31'
""").withColumn("month", F.date_format("month_date", "yyyy-MM"))

# For each month, calculate which subscriptions were active
mrr_monthly = df_subscriptions.alias("s").crossJoin(
    months.select("month").alias("m")
).filter(
    (F.col("m.month") >= F.date_format(F.col("s.subscription_start_date"), "yyyy-MM")) &
    (
        (F.col("s.status") == "active") |
        (
            (F.col("s.status") == "cancelled") & 
            (F.col("m.month") < F.date_format(F.col("s.cancel_date"), "yyyy-MM"))
        )
    )
).groupBy("m.month").agg(
    F.round(F.sum("s.mrr_usd"), 2).alias("mrr"),
    F.countDistinct("s.customer_id").alias("active_subscribers")
).withColumn("arr", F.round(F.col("mrr") * 12, 2)) \
 .orderBy("month")

mrr_monthly.createOrReplaceTempView("mrr_monthly")
mrr_monthly.display()

# COMMAND ----------

# --- MRR Movement (New, Expansion, Contraction, Churn) ---
# Simplified MRR waterfall

# New MRR by month (first subscription month)
new_mrr = df_subscriptions.withColumn(
    "start_month", F.date_format("subscription_start_date", "yyyy-MM")
).groupBy("start_month").agg(
    F.round(F.sum("mrr_usd"), 2).alias("new_mrr"),
    F.count("customer_id").alias("new_subscribers")
).withColumnRenamed("start_month", "month")

# Churned MRR by month
churned_mrr = df_subscriptions.filter(F.col("status") == "cancelled") \
    .withColumn("churn_month", F.date_format("cancel_date", "yyyy-MM")) \
    .groupBy("churn_month").agg(
        F.round(F.sum("mrr_usd"), 2).alias("churned_mrr"),
        F.count("customer_id").alias("churned_subscribers")
    ).withColumnRenamed("churn_month", "month")

mrr_movement = new_mrr.join(churned_mrr, "month", "outer") \
    .fillna(0) \
    .withColumn("net_new_mrr", F.col("new_mrr") - F.col("churned_mrr")) \
    .orderBy("month")

mrr_movement.createOrReplaceTempView("mrr_movement")
mrr_movement.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ### 💰 LTV Estimates by Segment

# COMMAND ----------

# LTV = ARPU * Avg Lifetime (months) * Gross Margin
# Assumption: ~70% gross margin for supplement products

GROSS_MARGIN = 0.70

# Calculate LTV by various segments
ltv_base = df_subscriptions.join(df_customers, "customer_id") \
    .join(df_orders.groupBy("customer_id").agg(
        F.round(F.sum("order_total_usd"), 2).alias("total_revenue_per_customer"),
        F.count("order_id").alias("total_orders_per_customer")
    ), "customer_id", "left")

# Overall LTV
overall_ltv = ltv_base.agg(
    F.round(F.avg("total_revenue_per_customer"), 2).alias("avg_revenue_per_customer"),
    F.round(F.avg("months_active"), 1).alias("avg_lifetime_months"),
    F.round(F.avg("mrr_usd"), 2).alias("avg_mrr")
).withColumn("estimated_ltv", F.round(F.col("avg_mrr") * F.col("avg_lifetime_months") * GROSS_MARGIN, 2))

print("📊 Overall LTV Estimate:")
overall_ltv.display()

# LTV by Product
ltv_by_product = ltv_base.groupBy("product_id").agg(
    F.count("customer_id").alias("customers"),
    F.round(F.avg("mrr_usd"), 2).alias("avg_mrr"),
    F.round(F.avg("months_active"), 1).alias("avg_lifetime_months"),
    F.round(F.avg("total_revenue_per_customer"), 2).alias("avg_total_revenue")
).withColumn("estimated_ltv", F.round(F.col("avg_mrr") * F.col("avg_lifetime_months") * GROSS_MARGIN, 2)) \
 .orderBy(F.desc("estimated_ltv"))

ltv_by_product.createOrReplaceTempView("ltv_by_product")
print("\n📊 LTV by Product:")
ltv_by_product.display()

# LTV by Channel
ltv_by_channel = ltv_base.groupBy("acquisition_channel").agg(
    F.count("customer_id").alias("customers"),
    F.round(F.avg("mrr_usd"), 2).alias("avg_mrr"),
    F.round(F.avg("months_active"), 1).alias("avg_lifetime_months"),
    F.round(F.avg("total_revenue_per_customer"), 2).alias("avg_total_revenue")
).withColumn("estimated_ltv", F.round(F.col("avg_mrr") * F.col("avg_lifetime_months") * GROSS_MARGIN, 2)) \
 .orderBy(F.desc("estimated_ltv"))

ltv_by_channel.createOrReplaceTempView("ltv_by_channel")
print("\n📊 LTV by Acquisition Channel:")
ltv_by_channel.display()

# LTV by Region
ltv_by_region = ltv_base.groupBy("region").agg(
    F.count("customer_id").alias("customers"),
    F.round(F.avg("mrr_usd"), 2).alias("avg_mrr"),
    F.round(F.avg("months_active"), 1).alias("avg_lifetime_months"),
    F.round(F.avg("total_revenue_per_customer"), 2).alias("avg_total_revenue")
).withColumn("estimated_ltv", F.round(F.col("avg_mrr") * F.col("avg_lifetime_months") * GROSS_MARGIN, 2)) \
 .orderBy(F.desc("estimated_ltv"))

ltv_by_region.createOrReplaceTempView("ltv_by_region")
print("\n📊 LTV by Region:")
ltv_by_region.display()

# LTV by Persona
ltv_by_persona = ltv_base.groupBy("customer_persona").agg(
    F.count("customer_id").alias("customers"),
    F.round(F.avg("mrr_usd"), 2).alias("avg_mrr"),
    F.round(F.avg("months_active"), 1).alias("avg_lifetime_months"),
    F.round(F.avg("total_revenue_per_customer"), 2).alias("avg_total_revenue")
).withColumn("estimated_ltv", F.round(F.col("avg_mrr") * F.col("avg_lifetime_months") * GROSS_MARGIN, 2)) \
 .orderBy(F.desc("estimated_ltv"))

ltv_by_persona.createOrReplaceTempView("ltv_by_persona")
print("\n📊 LTV by Customer Persona:")
ltv_by_persona.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 6: CUSTOMER BEHAVIOR & GROWTH OPPORTUNITIES
# MAGIC **Insight:** _Where should we invest to grow? Which segments are underleveraged?_

# COMMAND ----------

# --- Customer Acquisition Funnel by Month ---
acquisition_trend = df_customers.groupBy("acquisition_month").agg(
    F.count("customer_id").alias("new_customers")
).join(
    df_subscriptions.withColumn("start_month", F.date_format("subscription_start_date", "yyyy-MM")) \
        .groupBy("start_month").agg(
            F.count("subscription_id").alias("new_subscriptions")
        ).withColumnRenamed("start_month", "acquisition_month"),
    "acquisition_month",
    "left"
).withColumn(
    "conversion_rate", 
    F.round(F.col("new_subscriptions") / F.col("new_customers") * 100, 1)
).orderBy("acquisition_month")

acquisition_trend.createOrReplaceTempView("acquisition_trend")
acquisition_trend.display()

# COMMAND ----------

# --- Channel Performance Matrix ---
channel_performance = df_customers.join(
    df_subscriptions, "customer_id", "left"
).join(
    df_orders.groupBy("customer_id").agg(
        F.sum("order_total_usd").alias("total_spend")
    ), "customer_id", "left"
).groupBy("acquisition_channel").agg(
    F.count(F.col("customer_id")).alias("total_customers"),
    F.sum(F.when(F.col("status") == "active", 1).otherwise(0)).alias("active_subs"),
    F.sum(F.when(F.col("status") == "cancelled", 1).otherwise(0)).alias("churned_subs"),
    F.round(F.avg("months_active"), 1).alias("avg_tenure"),
    F.round(F.sum("total_spend"), 2).alias("total_revenue"),
    F.round(F.avg("total_spend"), 2).alias("avg_revenue_per_customer")
).withColumn(
    "active_rate", F.round(F.col("active_subs") / F.col("total_customers") * 100, 1)
).orderBy(F.desc("total_revenue"))

channel_performance.createOrReplaceTempView("channel_performance")
channel_performance.display()

# COMMAND ----------

# --- Geographic Opportunity Analysis ---
geo_analysis = df_customers.join(
    df_subscriptions, "customer_id", "left"
).groupBy("region", "country").agg(
    F.count("customer_id").alias("customers"),
    F.sum(F.when(F.col("status") == "active", 1).otherwise(0)).alias("active_subs"),
    F.round(F.sum(F.when(F.col("status") == "active", F.col("mrr_usd")).otherwise(0)), 2).alias("mrr"),
    F.round(F.avg("months_active"), 1).alias("avg_tenure")
).withColumn(
    "mrr_per_customer", F.round(F.col("mrr") / F.col("active_subs"), 2)
).orderBy(F.desc("mrr"))

geo_analysis.createOrReplaceTempView("geo_analysis")
geo_analysis.display()

# COMMAND ----------

# --- Product Mix & Upgrade Opportunity ---
product_mix = df_subscriptions.join(
    df_products.select("product_id", "product_name", "product_category"),
    "product_id"
).groupBy("product_name", "product_category", "status").agg(
    F.count("customer_id").alias("customers"),
    F.round(F.sum("mrr_usd"), 2).alias("total_mrr"),
    F.round(F.avg("months_active"), 1).alias("avg_tenure")
)

product_mix.createOrReplaceTempView("product_mix")
product_mix.display()

# Upgrade potential: Essentials users who could move to Beckham Stack
upgrade_candidates = churn_risk_scored.filter(
    (F.col("product_id") == "PROD_001") &
    (F.col("risk_level").isin("🟢 Low", "🟡 Medium")) &
    (F.col("months_active") >= 3) &
    (F.col("avg_engagement_last3m") > 50)
)

upgrade_potential_mrr = upgrade_candidates.agg(
    F.count("customer_id").alias("upgrade_candidates"),
    F.round(F.sum("mrr_usd"), 2).alias("current_mrr"),
    # If they upgrade to Beckham Stack (~$182.67), incremental MRR
).withColumn("potential_incremental_mrr", 
    F.round(F.col("upgrade_candidates") * (182.67 - 84.00), 2)
)

print("📊 Upgrade Opportunity (Essentials → Beckham Stack):")
upgrade_potential_mrr.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 7: CANCELLATION ANALYSIS
# MAGIC **Insight:** _Why are customers leaving and when? What can we do about it?_

# COMMAND ----------

# Cancellation reasons analysis
cancel_analysis = df_subscriptions.filter(F.col("status") == "cancelled") \
    .groupBy("cancel_reason").agg(
        F.count("customer_id").alias("cancellations"),
        F.round(F.sum("mrr_usd"), 2).alias("lost_mrr"),
        F.round(F.avg("months_active"), 1).alias("avg_tenure_at_cancel")
    ).withColumn(
        "pct_of_cancellations", 
        F.round(F.col("cancellations") / F.sum("cancellations").over(Window.partitionBy()), 1)
    ).orderBy(F.desc("cancellations"))

cancel_analysis.createOrReplaceTempView("cancellation_reasons")
cancel_analysis.display()

# Churn by tenure month (when do people churn?)
churn_timing = df_subscriptions.filter(F.col("status") == "cancelled") \
    .groupBy("months_active").agg(
        F.count("customer_id").alias("churned_customers"),
        F.round(F.sum("mrr_usd"), 2).alias("lost_mrr")
    ).orderBy("months_active")

churn_timing.createOrReplaceTempView("churn_timing")
churn_timing.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 8: ENGAGEMENT-RETENTION CORRELATION
# MAGIC **Insight:** _Quantify the relationship between engagement and retention to justify investment in the app experience._

# COMMAND ----------

# Engagement score buckets vs retention
engagement_retention = df_engagement.join(
    df_subscriptions.select("customer_id", "status", "months_active"),
    "customer_id",
    "inner"
).groupBy("customer_id", "status").agg(
    F.round(F.avg("engagement_score"), 0).alias("avg_engagement_score"),
    F.round(F.avg("app_opens"), 0).alias("avg_monthly_app_opens"),
    F.max("months_active").alias("tenure_months")
)

# Create engagement buckets
engagement_buckets = engagement_retention.withColumn(
    "engagement_bucket",
    F.when(F.col("avg_engagement_score") >= 80, "🟢 High (80-100)")
     .when(F.col("avg_engagement_score") >= 50, "🟡 Medium (50-79)")
     .when(F.col("avg_engagement_score") >= 25, "🟠 Low (25-49)")
     .otherwise("🔴 Very Low (0-24)")
)

engagement_impact = engagement_buckets.groupBy("engagement_bucket").agg(
    F.count("customer_id").alias("customers"),
    F.round(F.avg("tenure_months"), 1).alias("avg_tenure_months"),
    F.sum(F.when(F.col("status") == "active", 1).otherwise(0)).alias("active"),
    F.sum(F.when(F.col("status") == "cancelled", 1).otherwise(0)).alias("churned")
).withColumn(
    "retention_rate", F.round(F.col("active") / F.col("customers") * 100, 1)
).orderBy("engagement_bucket")

engagement_impact.createOrReplaceTempView("engagement_retention_impact")
engagement_impact.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 KPI 9: REFERRAL NETWORK & VIRAL COEFFICIENT

# COMMAND ----------

# Referral analysis
referral_stats = df_customers.filter(F.col("acquisition_channel") == "Referral") \
    .join(df_subscriptions, "customer_id", "left") \
    .agg(
        F.count("customer_id").alias("referred_customers"),
        F.sum(F.when(F.col("status") == "active", 1).otherwise(0)).alias("active_referred"),
        F.round(F.avg("months_active"), 1).alias("avg_tenure_referred")
    )

total_referrals_made = df_engagement.agg(F.sum("referrals_made")).collect()[0][0]
active_customer_base = df_subscriptions.filter(F.col("status") == "active").count()
viral_coefficient = round(total_referrals_made / active_customer_base, 3) if active_customer_base > 0 else 0

print(f"📊 Viral Coefficient: {viral_coefficient}")
print(f"📊 Total Referrals Generated: {total_referrals_made}")
referral_stats.display()

# Monthly referral trend
referral_trend = df_engagement.groupBy("month").agg(
    F.sum("referrals_made").alias("referrals")
).orderBy("month")

referral_trend.createOrReplaceTempView("referral_trend")
referral_trend.display()

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## 📊 SAVE ALL ANALYTICAL VIEWS FOR DASHBOARD

# COMMAND ----------

# Save key analytical views as Delta tables for the dashboard
analytical_views = {
    "kpi_cohort_retention": "cohort_retention",
    "kpi_cohort_retention_pivot": "cohort_retention_pivot",
    "kpi_churn_risk_summary": "churn_risk_summary",
    "kpi_churn_risk_scored": "churn_risk_scored",
    "kpi_churn_risk_by_product": "churn_risk_by_product",
    "kpi_churn_risk_by_channel": "churn_risk_by_channel",
    "kpi_churn_risk_by_region": "churn_risk_by_region",
    "kpi_churn_risk_by_persona": "churn_risk_by_persona",
    "kpi_churn_watchlist": "churn_watchlist",
    "kpi_retention_by_health_outcome": "retention_by_health_outcome",
    "kpi_health_trajectory": "health_trajectory",
    "kpi_product_health_efficacy": "product_health_efficacy",
    "kpi_mrr_monthly": "mrr_monthly",
    "kpi_mrr_movement": "mrr_movement",
    "kpi_ltv_by_product": "ltv_by_product",
    "kpi_ltv_by_channel": "ltv_by_channel",
    "kpi_ltv_by_region": "ltv_by_region",
    "kpi_ltv_by_persona": "ltv_by_persona",
    "kpi_acquisition_trend": "acquisition_trend",
    "kpi_channel_performance": "channel_performance",
    "kpi_geo_analysis": "geo_analysis",
    "kpi_product_mix": "product_mix",
    "kpi_cancellation_reasons": "cancellation_reasons",
    "kpi_churn_timing": "churn_timing",
    "kpi_engagement_retention": "engagement_retention_impact",
    "kpi_referral_trend": "referral_trend"
}

for table_name, view_name in analytical_views.items():
    spark.table(view_name).write.mode("overwrite").saveAsTable(f"{database_name}.{table_name}")
    count = spark.table(f"{database_name}.{table_name}").count()
    print(f"  ✅ {database_name}.{table_name}: {count:,} rows")

print(f"\n🎉 All {len(analytical_views)} analytical views saved to Delta Lake!")

✅ All tables loaded


context,metric,value
Since Dec 2024 launch,Total Customers Acquired,"2,800"
55.2% of total customers,Active Subscribers,"1,546"
Current active subscriptions,Monthly Recurring Revenue (MRR),"$192,620.23"
MRR × 12,Annualized Run Rate (ARR),"$2,311,442.76"
All orders to date,Total Revenue,"$1,584,221.10"
Across all order types,Average Order Value (AOV),$129.61
"959 of 2,505 subscriptions",Overall Churn Rate,38.3%


cohort,cohort_size,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
2024-12,67,100.0,83.6,71.6,61.2,53.7,52.2,50.7,49.3,47.8,47.8,44.8,41.8,41.8,41.8,14.9
2025-01,106,100.0,84.9,74.5,64.2,58.5,55.7,53.8,52.8,51.9,50.9,47.2,45.3,43.4,6.6,null
2025-02,102,100.0,82.4,73.5,70.6,63.7,60.8,59.8,57.8,57.8,55.9,52.9,52.9,5.9,null,null
2025-03,165,100.0,84.2,77.0,70.9,64.8,59.4,55.2,52.7,52.1,51.5,49.1,5.5,null,null,null
2025-04,171,100.0,82.5,76.6,68.4,59.1,55.6,53.8,50.9,49.1,47.4,9.9,null,null,null,null
2025-05,187,100.0,82.4,69.5,63.6,60.4,58.8,56.1,53.5,52.4,10.7,null,null,null,null,null
2025-06,217,100.0,85.7,75.6,70.0,65.0,59.4,56.2,55.3,8.8,null,null,null,null,null,null
2025-07,225,100.0,81.8,73.3,66.2,64.0,60.4,57.8,6.7,null,null,null,null,null,null,null
2025-08,224,100.0,82.6,70.1,64.7,61.2,58.9,4.5,null,null,null,null,null,null,null,null
2025-09,252,100.0,86.5,79.0,73.8,72.2,6.0,null,null,null,null,null,null,null,null,null


cohort,tenure_month,retained_customers,cohort_size,retention_rate
2024-12,1,67,67,100.0
2024-12,2,56,67,83.6
2024-12,3,48,67,71.6
2024-12,4,41,67,61.2
2024-12,5,36,67,53.7
2024-12,6,35,67,52.2
2024-12,7,34,67,50.7
2024-12,8,33,67,49.3
2024-12,9,32,67,47.8
2024-12,10,32,67,47.8


risk_level,customer_count,mrr_at_risk,avg_tenure_months,avg_engagement
🟢 Low,1273,158816.23,6.0,56.8
🟡 Medium,250,30764.34,6.5,51.3
🟠 High,22,2955.66,9.7,41.9
🔴 Critical,1,84.0,12.0,36.9


...

engagement_bucket,customers,avg_tenure_months,active,churned,retention_rate
🔴 Very Low (0-24),542,1.6,0,542,0.0
🟠 Low (25-49),641,6.2,296,345,46.2
🟡 Medium (50-79),1288,5.5,1216,72,94.4
🟢 High (80-100),34,2.3,34,0,100.0


📊 Viral Coefficient: 0.29
📊 Total Referrals Generated: 448


referred_customers,active_referred,avg_tenure_referred
275,164,5.1


month,referrals
2024-12,0
2025-01,5
2025-02,7
2025-03,12
2025-04,21
2025-05,27
2025-06,25
2025-07,38
2025-08,46
2025-09,43


  ✅ im8_health.kpi_cohort_retention: 118 rows
  ✅ im8_health.kpi_cohort_retention_pivot: 14 rows
  ✅ im8_health.kpi_churn_risk_summary: 4 rows
  ✅ im8_health.kpi_churn_risk_scored: 1,546 rows
  ✅ im8_health.kpi_churn_risk_by_product: 10 rows
  ✅ im8_health.kpi_churn_risk_by_channel: 26 rows
  ✅ im8_health.kpi_churn_risk_by_region: 10 rows
  ✅ im8_health.kpi_churn_risk_by_persona: 16 rows
  ✅ im8_health.kpi_churn_watchlist: 20 rows
  ✅ im8_health.kpi_retention_by_health_outcome: 3 rows
  ✅ im8_health.kpi_health_trajectory: 28 rows
  ✅ im8_health.kpi_product_health_efficacy: 3 rows
  ✅ im8_health.kpi_mrr_monthly: 14 rows
  ✅ im8_health.kpi_mrr_movement: 14 rows
  ✅ im8_health.kpi_ltv_by_product: 3 rows
  ✅ im8_health.kpi_ltv_by_channel: 9 rows
  ✅ im8_health.kpi_ltv_by_region: 3 rows
  ✅ im8_health.kpi_ltv_by_persona: 6 rows
  ✅ im8_health.kpi_acquisition_trend: 14 rows
  ✅ im8_health.kpi_channel_performance: 9 rows
  ✅ im8_health.kpi_geo_analysis: 12 rows
  ✅ im8_health.kpi_product_mix: